In [1]:
import sys, os
import numpy as np
import pandas as pd
from utils.utils import read_txt, read_txt_np_int
from functools import lru_cache

# INPUT

In [2]:
inputfilename = './inputs/day13A.txt'

inputdata = read_txt(inputfilename)

testdata = ["Button A: X+94, Y+34",
"Button B: X+22, Y+67",
"Prize: X=8400, Y=5400",

"Button A: X+26, Y+66",
"Button B: X+67, Y+21",
"Prize: X=12748, Y=12176",

"Button A: X+17, Y+86",
"Button B: X+84, Y+37",
"Prize: X=7870, Y=6450",

"Button A: X+69, Y+23",
"Button B: X+27, Y+71",
"Prize: X=18641, Y=10279"]

In [6]:
data_to_use = inputdata

claw_machines = []
for line in data_to_use:
    if "Button A: " in line:
        buttonA = np.array([int(x.split('+')[1]) for x in line.split(': ')[1].split(', ')])
    if "Button B: " in line:
        buttonB = np.array([int(x.split('+')[1]) for x in line.split(': ')[1].split(', ')])
    if "Prize: " in line:
        prize = np.array([int(x.split('=')[1]) for x in line.split(': ')[1].split(', ')])
        claw_machines.append({'ButtonA': buttonA, 'ButtonB': buttonB, 'Prize': prize})

print(len(claw_machines))
claw_machines

320


[{'ButtonA': array([75, 30]),
  'ButtonB': array([57, 75]),
  'Prize': array([9885, 8130])},
 {'ButtonA': array([48, 12]),
  'ButtonB': array([20, 52]),
  'Prize': array([1320, 5260])},
 {'ButtonA': array([20, 86]),
  'ButtonB': array([46, 33]),
  'Prize': array([370, 767])},
 {'ButtonA': array([21, 77]),
  'ButtonB': array([65, 36]),
  'Prize': array([4616, 8832])},
 {'ButtonA': array([49, 88]),
  'ButtonB': array([90, 36]),
  'Prize': array([3292, 5284])},
 {'ButtonA': array([46, 13]),
  'ButtonB': array([11, 71]),
  'Prize': array([6610, 7336])},
 {'ButtonA': array([55, 23]),
  'ButtonB': array([25, 47]),
  'Prize': array([8960, 6506])},
 {'ButtonA': array([19, 90]),
  'ButtonB': array([91, 28]),
  'Prize': array([1245, 3076])},
 {'ButtonA': array([77, 15]),
  'ButtonB': array([55, 57]),
  'Prize': array([7535, 2301])},
 {'ButtonA': array([29, 59]),
  'ButtonB': array([39, 11]),
  'Prize': array([15636,  3044])},
 {'ButtonA': array([17, 94]),
  'ButtonB': array([68, 34]),
  'Prize':

## PART 1

In [14]:
def check_machine(prize, buttonA, buttonB):
    buttonA_delta = buttonA[1] - buttonA[0]
    buttonB_delta = buttonB[1] - buttonB[0]
    prize_delta = prize[1] - prize[0]

    buttonA_ratio = buttonA[1]/buttonA[0]
    buttonB_ratio = buttonB[1]/buttonB[0]
    prize_ratio = prize[1]/prize[0]

    if (buttonA_delta*buttonB_delta <= 0):
        return
    else:
        print(f"Prize: {prize}, ButtonA: {buttonA}, ButtonB: {buttonB}")
        print(f"Prize ratio: {prize_ratio}, ButtonA ratio: {buttonA_ratio}, ButtonB ratio: {buttonB_ratio}")

def get_valid_presses(prize, buttonA, buttonB, max_presses):
    valid_presses = []
    # We keep removing buttonA and check buttonB
    for i in range(max_presses+1):
        pending_distance = prize - i*buttonA
        if (pending_distance < 0).any():
            break
        ratio0 = pending_distance[0] / buttonB[0]
        ratio1 = pending_distance[1] / buttonB[1]
        if (ratio0 == ratio1) and ratio0 <= max_presses and ratio0.is_integer():
            valid_presses.append([i, ratio0])
    return valid_presses

def solve_machine(prize, buttonA, buttonB, max_presses, press_token_cost):
    valid_presses = get_valid_presses(prize, buttonA, buttonB, max_presses)
    if len(valid_presses) == 0:
        return None
    lowest_cost = max_presses*press_token_cost[0] + max_presses*press_token_cost[1]
    for valid_press in valid_presses:
        token_cost = valid_press[0]*press_token_cost[0] + valid_press[1]*press_token_cost[1]
        if token_cost < lowest_cost:
            lowest_cost = token_cost
    return lowest_cost


In [17]:
# Solve part 1
total_token_cost = 0
all_token_costs_1 = []
for claw_machine in claw_machines:
    token_cost = solve_machine(claw_machine['Prize'], claw_machine['ButtonA'], claw_machine['ButtonB'], 100, (3, 1))
    all_token_costs_1.append(token_cost)
    if token_cost:
        total_token_cost += token_cost
print(int(total_token_cost))

33481


## PART 2

In [30]:
def solve_linear_system(prize, buttonA, buttonB):
    A = np.array([buttonA, buttonB]).transpose()
    if np.linalg.det(A) == 0:
        print(f"Multiple solutions for {prize}: buttonA: {buttonA}, buttonB: {buttonB}")
        return None
    B = prize
    X = np.linalg.solve(A, B)
    X_int = X.round(decimals=0).astype(int)
    if (np.dot(A, X_int) == B).all():
        return X_int
    return None

def get_cost(prize, buttonA, buttonB, press_token_cost):
    presses = solve_linear_system(prize, buttonA, buttonB)
    if presses is not None:
        return np.dot(presses, press_token_cost)
    return None

In [ ]:
# Solve part 2
total_token_cost = 0
all_token_costs_2 = []
for claw_machine in claw_machines:
    prize = claw_machine['Prize']+ np.array([10000000000000,10000000000000])
    buttonA = claw_machine['ButtonA']
    buttonB = claw_machine['ButtonB']
    token_cost = get_cost(prize, buttonA, buttonB, np.array([3, 1]))
    all_token_costs_2.append(token_cost)
    if token_cost is not None:
        total_token_cost += token_cost
print(total_token_cost)

92572057880885
